In [1]:
import torch
import random
import cv2
import numpy as np
from torch import nn
from torchvision import models
from torch.utils import data
from matplotlib import pyplot as plt
from torch.nn import functional as F
%matplotlib inline

In [2]:
x = np.random.randn(3, 5)
x

array([[ 0.32009542, -1.0410011 , -0.45408841,  1.15025677,  0.99723141],
       [ 0.03599228,  1.0898217 , -1.17736566, -1.21532241, -1.63121653],
       [-0.57987694,  2.34898176, -0.46289087,  1.14147415, -0.61507659]])

In [4]:
y = x
x[:, 0] = 1.2
x, y

(array([[1.2       , 1.08368329, 0.20619628, 1.32309065, 0.99447048],
        [1.2       , 1.18771134, 1.3861899 , 1.47700856, 2.66086736],
        [1.2       , 5.5177153 , 0.21426796, 1.30296323, 0.37831921]]),
 array([[1.2       , 1.08368329, 0.20619628, 1.32309065, 0.99447048],
        [1.2       , 1.18771134, 1.3861899 , 1.47700856, 2.66086736],
        [1.2       , 5.5177153 , 0.21426796, 1.30296323, 0.37831921]]))

In [3]:
x[:, 0].sort()
x

array([[-0.60736245,  0.05312085,  1.88691183, -0.72999724, -0.99155226],
       [ 0.20374399,  0.30048661, -0.41601636, -0.53726804,  0.43218936],
       [ 0.45909998, -1.62834234, -0.55385048, -0.33428451,  0.74530595]])

In [4]:
plt.rcParams["axes.unicode_minus"] = False
plt.figure()
plt.plot(range(3), x, label=["a", "b", "c", "d", "e"])
plt.legend(frameon=False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
a = torch.tensor(10., requires_grad=True)
b = torch.tensor(4.)
c = torch.tensor(5., requires_grad=True)
f = lambda x, y , c : x * y + c
for i in range(4):
    l = f(a, b, c)
    l.backward()
    # with torch.no_grad():
    #     a += a

In [2]:
input = torch.randn(5, 3, 4, 4)
target = torch.randint(3, (5, 4, 4))
F.cross_entropy(input, target, reduction="none")

tensor([[[0.5399, 0.1556, 1.6685, 2.0749],
         [0.2134, 1.5019, 0.7016, 1.1997],
         [0.7786, 1.5289, 0.6386, 1.3042],
         [3.7195, 3.2603, 0.5434, 1.1997]],

        [[0.8360, 0.2738, 1.1483, 0.4526],
         [1.8703, 2.1785, 3.0973, 0.2953],
         [2.0495, 0.2592, 0.9236, 1.2743],
         [0.9476, 2.0289, 1.9265, 0.5526]],

        [[0.0885, 0.5400, 1.7490, 1.0004],
         [1.2619, 0.5605, 0.9871, 0.5597],
         [1.8031, 1.3143, 1.7468, 2.7397],
         [0.9348, 1.8272, 0.2260, 0.5582]],

        [[0.7739, 1.5960, 2.1234, 1.3273],
         [0.7942, 0.9123, 1.9258, 2.7585],
         [2.5981, 2.7038, 0.3616, 0.2986],
         [1.0910, 2.3436, 0.7527, 1.1329]],

        [[0.2778, 1.2651, 1.4387, 2.8061],
         [1.2079, 1.2134, 3.3150, 1.2151],
         [0.7080, 2.4715, 0.4660, 1.5311],
         [0.2506, 1.4355, 1.0796, 0.7543]]])

In [3]:
# 定义 bilinear kernel
def bilinear_kernel(in_channels, out_channels, kernel_size):
    """
    return a bilinear filter tensor
    """
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * (1 - abs(og[1] - center) / factor)
    weight = np.zeros(
        (in_channels, out_channels, kernel_size, kernel_size), dtype="float32"
    )
    weight[range(in_channels), range(out_channels), :, :] = filt
    return torch.from_numpy(weight)

In [4]:
bilinear_kernel(1, 1, 4)

tensor([[[[0.0625, 0.1875, 0.1875, 0.0625],
          [0.1875, 0.5625, 0.5625, 0.1875],
          [0.1875, 0.5625, 0.5625, 0.1875],
          [0.0625, 0.1875, 0.1875, 0.0625]]]])

In [5]:
x = torch.randn(3, 4)
x, x[:, :2], x[:, None, :2], torch.max(x[:, None, :2], x[:, :2])
torch.argmax(x)

tensor(6)

In [6]:
x = torch.arange(20).reshape(2, 5, 2)
x.sum(dim=2)
torch.zeros(20, dtype=torch.long)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
def conv_trans(input, kernel):
    h, w = kernel.shape
    Y = torch.zeros(input.shape[0] + h - 1, input.shape[1] + w - 1)
    for i in range(input.shape[0]):
        for j in range(input.shape[1]):
            Y[i : i + h, j : j + w] += kernel * input[i, j]
    return Y

In [8]:
input = torch.arange(16).reshape(4, 4)
kernel = torch.randn(3, 3)
conv_trans(input, kernel)

tensor([[  0.0000,  -0.7475,  -1.5456,  -2.4634,  -0.3913,  -0.3590],
        [ -2.9900,  -5.7411,  -9.9993, -15.6584,  -8.1336,  -6.1172],
        [-13.1840, -22.9570, -36.6328, -45.3979, -22.3950, -17.2545],
        [-31.7679, -45.6735, -71.6931, -80.4582, -38.8714, -29.5984],
        [-38.3919, -54.8722, -90.5229, -98.3702, -52.2316, -39.6686],
        [-25.1699, -24.8917, -41.2687, -44.3746, -13.9208, -18.0968]])

In [9]:
def sythentic_data(w, b, num):
    x = torch.normal(0, 2, size=(num, len(w)))
    y = x @ w + b + torch.randn(num)
    return x, y

In [10]:
def load_data(x, y, batch_size):
    n = len(x)
    indices = list(range(n))
    random.shuffle(indices)
    for i in range(0, n, batch_size):
        id = indices[i:min(i + batch_size, n)]
        yield x[id], y[id]

In [11]:
def mseloss(x, y, w, b):
    h = x @ w + b
    return torch.sum(torch.pow(h - y, 2)) / (2 * len(x))

In [12]:
def gradientDescent(*params, lr):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad
            param.grad.zero_()

In [13]:
true_w = torch.tensor([2, 1.5])
true_b = torch.tensor(3)
num = 256
batch_size =20
X, Y = sythentic_data(true_w, true_b, num)
w = torch.randn(2, requires_grad=True)
b = torch.zeros(1, requires_grad=True, dtype=torch.float)
epoch = 40
lr = 0.01
feature, label = next(load_data(X, Y, batch_size))
mseloss(feature, label, w, b)

tensor(41.4106, grad_fn=<DivBackward0>)

In [14]:
for _ in range(epoch):
    for f, l in load_data(X, Y, batch_size):
        mseloss(f, l, w, b).backward()
        gradientDescent(w, b, lr=lr)

In [16]:
w = w - w.grad
id(w), id(w.grad)

TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

In [16]:
w -= w.grad # requires_grad=True不能执行替换操作，因为张量保存的梯度与张量值有关，所以pytorch会报错
# w.grad.zero_()
type(w)

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [ ]:
x1 = torch.arange(6.).reshape(2, 3)
x2 = torch.arange(6., 12., requires_grad=True).reshape(2, 3)
y = x1 * x2
y.backward(gradient=torch.ones(y.shape))
x1, x2, torch.stack([x1, x2], dim=0)

In [ ]:
net = nn.Sequential(nn.Conv2d(6, 16, 5, 5), nn.Linear(16 * 5 * 5, 20)) # 矩阵形状为16*6*5*5
net.state_dict()

In [ ]:
X = np.random.randint(1, 4, (2, 3, 2, 2))                                                                                                                
X, X.mean(axis=(0, 2, 3), keepdims=True), X.mean(axis=1)

In [ ]:
def batch_norm(x, gamma, beta, e):
    var = x.var(dim=(0, 2, 3), keepdim=True)
    x_hat = (x - x.mean(dim=(0, 2, 3), keepdim=True)) / (var + e)
    y = gamma * x_hat + beta
    return y

In [ ]:
a = torch.rand(4, 2, 3, 2)
gamma = torch.ones(a.shape)
beta = torch.zeros(a.shape)
e = 0.001
batch_norm(a, gamma, beta, e)

In [ ]:
a = torch.arange(15).reshape(5, 3)
b = torch.arange(3).reshape(1, 3)
a + b

In [ ]:
prenet = models.vgg19()
prenet.features

In [ ]:
a = torch.randint(1, 4, (2, 3, 2, 2))
b = torch.randint(5, 8, (2, 3, 2, 2))
torch.cat([a, b], dim=1)

In [ ]:
c = torch.rand(3, 3) 
d = torch.rand(3, 3)
c[:, 0], torch.stack((c[:, 0], c[:, 1], d[:, 0], d[:, 1]), dim=0)

In [ ]:
b = torch.arange(4).reshape(2, 2)
torch.repeat_interleave(b, 2, dim=0)

In [ ]:
# @save
def multibox_prior(data, sizes, ratios):
    """生成以每个像素为中心具有不同形状的锚框。"""
    in_height, in_width = data.shape[-2:]
    device, num_sizes, num_ratios = data.device, len(sizes), len(ratios)
    boxes_per_pixel = num_sizes + num_ratios - 1
    size_tensor = torch.tensor(sizes, device=device)
    ratio_tensor = torch.tensor(ratios, device=device)

    # 为了将锚点移动到像素的中心，需要设置偏移量。
    # 因为一个像素的的高为1且宽为1，我们选择偏移我们的中心0.5
    offset_h, offset_w = 0.5, 0.5
    steps_h = 1.0 / in_height  # Scaled steps in y axis
    steps_w = 1.0 / in_width  # Scaled steps in x axis

    # 生成锚框的所有中心点
    center_h = (torch.arange(in_height, device=device) + offset_h) * steps_h
    center_w = (torch.arange(in_width, device=device) + offset_w) * steps_w
    shift_y, shift_x = torch.meshgrid(center_h, center_w)
    shift_y, shift_x = shift_y.reshape(-1), shift_x.reshape(-1)

    # 生成“boxes_per_pixel”个高和宽，
    # 之后用于创建锚框的四角坐标 (xmin, xmax, ymin, ymax)
    w = (
        torch.cat(
            (
                size_tensor * torch.sqrt(ratio_tensor[0]),
                sizes[0] * torch.sqrt(ratio_tensor[1:]),
            )
        )
        * in_height
        / in_width
    )  # Handle rectangular inputs
    h = torch.cat(
        (
            size_tensor / torch.sqrt(ratio_tensor[0]),
            sizes[0] / torch.sqrt(ratio_tensor[1:]),
        )
    )
    # 除以2来获得半高和半宽
    anchor_manipulations = (
        torch.stack((-w, -h, w, h)).T.repeat(in_height * in_width, 1) / 2
    )

    # 每个中心点都将有“boxes_per_pixel”个锚框，
    # 所以生成含所有锚框中心的网格，重复了“boxes_per_pixel”次
    out_grid = torch.stack(
        [shift_x, shift_y, shift_x, shift_y], dim=1
    ).repeat_interleave(boxes_per_pixel, dim=0)
    output = out_grid + anchor_manipulations
    return output.unsqueeze(0)


In [ ]:
img = plt.imread(r'D:\WorkSpace\PythonProject\pytorch\img\catdog.jpg')
h, w = img.shape[:2]

print(h, w)
X = torch.rand(size=(2, 3, h, w))
Y = multibox_prior(X, sizes=[0.75, 0.5, 0.25], ratios=[1, 2, 0.5])
Y.shape